### In this file, I just want to add some feature columns to the data that could be useful in our analysis in the future: 

<i> Data: Comes from the MTA site and only has data from the 6 line.  However the methods below can be applied there </i>
1. Day of week
1. Time of day (Morning, Afternoon, Evening, Late)
1. Create a better column that represents the actual number of entries (Difficult!!)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df_raw = pd.read_csv('Turnstile_Usage_Data__2018.csv')
df_raw.head()

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits
0,R232A,R176,03-06-00,33 ST,6,IRT,12/23/2017,20:00:00,REGULAR,102417472,68072550
1,R409,R449,01-00-02,E 149 ST,6,IRT,12/23/2017,08:00:00,REGULAR,1985376,6042304
2,R256,R182,00-00-02,116 ST,6,IRT,12/23/2017,04:00:00,REGULAR,765700,471312
3,R246,R177,00-03-02,68ST-HUNTER CO,6,IRT,12/23/2017,20:00:00,REGULAR,14882041,10581878
4,R227,R131,00-00-05,23 ST,6,IRT,12/23/2017,08:00:00,REGULAR,1524615,537740


In [4]:
df_working = df_raw.copy()

### 1. Add a weekday Column

In [5]:
def Add_Weekday(data_frame, column):
    dmap = {0:'Mon', 1: 'Tue', 2: 'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
    #Convert column to date
    data_frame[column] = pd.to_datetime(data_frame[column])
    #Add column which shows the Weekday in integer
    data_frame['Day_Number'] = data_frame[column].apply(lambda x: x.dayofweek)
    #Add column which shows the Weekday in words
    data_frame['Weekday'] = data_frame['Day_Number'].map(dmap)
    return data_frame

In [6]:
df_working = Add_Weekday(df_working, 'Date')

In [176]:
df_working.sample(5)

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits,Day_Number,Weekday,Hour,Time_of_Day,unique_ident
221128,R231,R176,00-00-00,33 ST,6,IRT,2018-05-10,09:00:00,REGULAR,5014550,3820571,3,Thu,9,Worktime,R231R17600-00-00
514457,R423,R429,00-03-01,PELHAM BAY PARK,6,IRT,2018-11-21,15:00:00,REGULAR,16849878,9718802,2,Wed,15,Worktime,R423R42900-03-01
393131,R407,R448,01-00-01,E 143/ST MARY'S,6,IRT,2018-08-22,09:00:00,REGULAR,206540,532465,2,Wed,9,Worktime,R407R44801-00-01
269137,R406,R448,00-00-00,E 143/ST MARY'S,6,IRT,2018-06-08,13:00:00,REGULAR,2520421,1317154,4,Fri,13,Worktime,R406R44800-00-00
406843,R252,R180,00-03-00,103 ST,6,IRT,2018-08-30,09:00:00,REGULAR,244513,317397,3,Thu,9,Worktime,R252R18000-03-00


### 2. Add a Time of Day column

In [181]:
print(type(df_working['Time'][0]))
#print(df_working['Time'].unique()) #There are several times of day present

<class 'str'>


In [178]:
def DayTime(data_frame, column):
    #Extract the Hour from the Time Column
    data_frame['Hour'] = data_frame[column].apply(lambda x: int(x.split(':')[0]))
    #Use the Hour column to create a Column which distinguishes time of day
    data_frame.loc[(data_frame['Hour'] >= 8) & (data_frame['Hour'] <= 18), 'Time_of_Day'] = 'Worktime'
    data_frame.loc[(data_frame['Hour'] > 18) & (data_frame['Hour'] <= 24), 'Time_of_Day'] = 'Evening'
    data_frame.loc[(data_frame['Hour'] < 8), 'Time_of_Day'] = 'Late'
    
    return data_frame

In [179]:
df_working = DayTime(df_working, 'Time')

In [180]:
df_working.sample(5)

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits,Day_Number,Weekday,Hour,Time_of_Day,unique_ident
158620,R405,R447,01-00-01,CYPRESS AV,6,IRT,2018-04-03,05:00:00,REGULAR,679602,955625,1,Tue,5,Late,R405R44701-00-01
120280,R421,R427,00-06-01,MIDDLETOWN RD,6,IRT,2018-03-04,16:00:00,REGULAR,0,196,6,Sun,16,Worktime,R421R42700-06-01
365867,R408,R449,00-00-02,E 149 ST,6,IRT,2018-08-05,21:00:00,REGULAR,4890859,2796869,6,Sun,21,Evening,R408R44900-00-02
323385,R226,R131,02-00-00,23 ST,6,IRT,2018-07-10,17:00:00,REGULAR,1998541024,1545580931,1,Tue,17,Worktime,R226R13102-00-00
562573,R227A,R131,01-00-00,23 ST,6,IRT,2018-12-21,04:00:00,REGULAR,273988,161113,4,Fri,4,Late,R227AR13101-00-00


### 3: Figure out how to get a starting point for Entries (Unsure of this method!):
1. Get starting point at August 1st 00:00:00 for every station
1. Create a unique Identifier column which is basically combo of CA, Unit and SCP
1. Create a new DF (df_init) with just the columns of Entries and that unique Identifier
1. Merge that df_init and the main df_working dataframe
1. Create a column which takes the difference between the Cumulative Entries column and the Initial Entries column from

In [155]:
df_aug = df_working.copy()


In [156]:
#Create a unique identifier column in both DF's that I can later use to join
df_aug['unique_ident'] = df_aug['C/A'] + df_aug['Unit']+df_aug['SCP'] 
df_working['unique_ident']= df_working['C/A'] + df_working['Unit']+df_working['SCP'] 

In [191]:
#df_aug['unique_ident'].value_counts().sort_values

In [192]:
df_aug.head()

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries,Exits,Day_Number,Weekday,Hour,Time_of_Day,unique_ident
358685,R252,R180,00-00-01,103 ST,6,IRT,2018-08-01,09:34:19,REGULAR,3663282,1455885,2,Wed,9,Worktime,R252R18000-00-01
358686,R410,R450,00-00-00,LONGWOOD AV,6,IRT,2018-08-01,09:00:00,REGULAR,5156459,1293659,2,Wed,9,Worktime,R410R45000-00-00
358687,R242,R049,01-00-02,51 ST,6,IRT,2018-08-01,08:00:00,REGULAR,762013,538016,2,Wed,8,Worktime,R242R04901-00-02
358688,R409,R449,01-00-02,E 149 ST,6,IRT,2018-08-01,21:00:00,REGULAR,2053587,6297494,2,Wed,21,Evening,R409R44901-00-02
358689,R247,R178,01-03-02,77 ST,6,IRT,2018-08-01,13:00:00,REGULAR,1616730,14169546,2,Wed,13,Worktime,R247R17801-03-02


In [193]:
#Only get the data from August 1st which will represent the starting point
df_aug = df_aug[(df_aug['Date'] == '2018-08-01')]

In [194]:
#Since there is multiple data for each day, I will only keep the Eariest Datapoint
df_aug = df_aug.drop_duplicates(subset=['unique_ident'], keep='first')

In [195]:
#Good to go!
df_aug['unique_ident'].value_counts().sort_values

<bound method Series.sort_values of R420R10700-00-00    1
R406R44800-00-00    1
R414R16200-03-00    1
R242R04901-03-03    1
R412R14600-06-01    1
                   ..
R416R24500-03-01    1
R409R44901-00-02    1
R246R17700-03-02    1
R248R17800-00-03    1
R419R32600-03-00    1
Name: unique_ident, Length: 264, dtype: int64>

In [163]:
df_init = df_aug[['Entries', 'unique_ident']].copy()

### My Code works after this, however I was wondering if there was a more elegant way to do this??

In [165]:
df_join = df_working.merge(df_init, on='unique_ident')

In [171]:
df_join['Entries_Difference'] = df_join['Entries_x'] - df_join['Entries_y']

In [186]:
#This should eliminate all the dates before August 1st and before October 1. 
df_join = df_join[(df_join['Date']> '2018-08-01') & (df_join['Date']< '2018-10-01') ]
df_join.sample(10)

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries_x,Exits,Day_Number,Weekday,Hour,Time_of_Day,unique_ident,Entries_y,Entries_Difference
76682,R405,R447,01-00-02,CYPRESS AV,6,IRT,2018-08-24,09:00:00,REGULAR,1050977,3534925,4,Fri,9,Worktime,R405R44701-00-02,1048151,2826
91862,R407,R448,01-00-00,E 143/ST MARY'S,6,IRT,2018-09-12,05:00:00,REGULAR,438937,774322,2,Wed,5,Late,R407R44801-00-00,437103,1834
296135,R421,R427,00-06-01,MIDDLETOWN RD,6,IRT,2018-09-28,09:00:00,REGULAR,0,214,4,Fri,9,Worktime,R421R42700-06-01,0,0
102683,R412,R146,00-06-00,HUNTS POINT AV,6,IRT,2018-09-13,01:00:00,REGULAR,1501,0,3,Thu,1,Late,R412R14600-06-00,1470,31
265939,R243,R049,00-03-03,51 ST,6,IRT,2018-09-16,08:00:00,REGULAR,6989415,4060356,6,Sun,8,Worktime,R243R04900-03-03,6954715,34700
139060,R413,R325,00-03-01,WHITLOCK AV,6,IRT,2018-08-07,16:00:00,REGULAR,4723806,2837937,1,Tue,16,Worktime,R413R32500-03-01,4718406,5400
281029,R403,R446,01-00-00,BROOK AV,6,IRT,2018-09-21,21:00:00,REGULAR,2814159,9163652,4,Fri,21,Evening,R403R44601-00-00,2799528,14631
76723,R405,R447,01-00-02,CYPRESS AV,6,IRT,2018-08-30,21:00:00,REGULAR,1051791,3538067,3,Thu,21,Evening,R405R44701-00-02,1048151,3640
156014,R410,R450,00-00-01,LONGWOOD AV,6,IRT,2018-09-09,05:00:00,REGULAR,893675,198282,6,Sun,5,Late,R410R45000-00-01,869780,23895
108911,R417,R222,00-00-02,PARKCHESTER,6,IRT,2018-08-04,05:00:00,REGULAR,7924236,2478129,5,Sat,5,Late,R417R22200-00-02,7915920,8316


In [197]:
#Below is an example of a working DF using a random Turnstile as an Example
df_join[df_join['unique_ident'] =='R411R45001-00-02'].sort_values(by=['Date', 'Hour']).head(20)

,C/A,Unit,SCP,Station,Line Name,Division,Date,Time,Description,Entries_x,Exits,Day_Number,Weekday,Hour,Time_of_Day,unique_ident,Entries_y,Entries_Difference
85191,R411,R450,01-00-02,LONGWOOD AV,6,IRT,2018-08-02,01:00:00,REGULAR,2433164,3199654,3,Thu,1,Late,R411R45001-00-02,2433155,9
85192,R411,R450,01-00-02,LONGWOOD AV,6,IRT,2018-08-02,05:00:00,REGULAR,2433165,3199687,3,Thu,5,Late,R411R45001-00-02,2433155,10
85187,R411,R450,01-00-02,LONGWOOD AV,6,IRT,2018-08-02,09:00:00,REGULAR,2433196,3199835,3,Thu,9,Worktime,R411R45001-00-02,2433155,41
85190,R411,R450,01-00-02,LONGWOOD AV,6,IRT,2018-08-02,13:00:00,REGULAR,2433207,3199922,3,Thu,13,Worktime,R411R45001-00-02,2433155,52
85188,R411,R450,01-00-02,LONGWOOD AV,6,IRT,2018-08-02,17:00:00,REGULAR,2433240,3200058,3,Thu,17,Worktime,R411R45001-00-02,2433155,85
85189,R411,R450,01-00-02,LONGWOOD AV,6,IRT,2018-08-02,21:00:00,REGULAR,2433269,3200236,3,Thu,21,Evening,R411R45001-00-02,2433155,114
85195,R411,R450,01-00-02,LONGWOOD AV,6,IRT,2018-08-03,01:00:00,REGULAR,2433276,3200346,4,Fri,1,Late,R411R45001-00-02,2433155,121
85196,R411,R450,01-00-02,LONGWOOD AV,6,IRT,2018-08-03,05:00:00,REGULAR,2433277,3200377,4,Fri,5,Late,R411R45001-00-02,2433155,122
85194,R411,R450,01-00-02,LONGWOOD AV,6,IRT,2018-08-03,09:00:00,REGULAR,2433309,3200524,4,Fri,9,Worktime,R411R45001-00-02,2433155,154
85197,R411,R450,01-00-02,LONGWOOD AV,6,IRT,2018-08-03,13:00:00,REGULAR,2433333,3200624,4,Fri,13,Worktime,R411R45001-00-02,2433155,178
